In [1]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, Queue
import random
import math
import time

In [15]:
class Agent:
    def __init__(self, numberOfCurrencies, sess, timeFrame=50,  initialPortfolio=10000.0):
        self._s = sess
        self._isTraining = tf.placeholder(dtype=bool)
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.model = tf.layers.conv2d(inputs=self.inputT, filters=3, kernel_size=[1,3], activation=tf.nn.leaky_relu)
        self.model = tf.layers.conv2d(inputs=self.model, filters=20, kernel_size=[1,48], activation=tf.nn.leaky_relu)
        self.model = tf.layers.dense(self.model, 1000, activation=tf.nn.leaky_relu)
        self.model = tf.layers.dense(self.model, 1)
        self._allocate = tf.nn.softmax(self.model, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))
        #self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(tf.reshape(self.priceChanges, [-1, numberOfCurrencies])), self._allocate), 1))
        print(self.priceChanges.shape)
        print(self._allocate.shape)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices,
                      self._isTraining: True
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run([self.averageLoss, self._allocate], feed_dict={self.inputT : obs, self.priceChanges: prices, self._isTraining: False}))
        print(prices)

In [3]:
class DataManager:
    def __init__(self, batch_size=1, batch_count=0, period_size=50, batch_offset=0, thread_count=8):
        self.batchPath = os.path.abspath("./Batches")
        self.labelPath = os.path.abspath("./Labels")
        self.batch = batch_count
        self.batch_size = batch_size
        self.period_size = period_size
        self.offset = batch_offset
        self.thread_count = thread_count
        
            
    def importData(self, simulator):
        testSim = simulator
        PERIOD_SIZE = self.period_size
        BATCH_SIZE = self.batch_size
        
        if self.batch == 0:
            minSize = math.inf
            for pair in simulator.df.values():
                if len(pair.index) < minSize:
                    minSize = len(pair.index)
                print(len(pair.index))
            self.batch = int(math.floor(minSize / self.batch_size)) - 1
            print(self.batch)

        BATCH_COUNT = self.batch
        BATCH_OFFSET = self.offset
        
        activeProcess = 0
        processes = []
        index = 0
        count = 0
        while count != BATCH_COUNT:
            while (activeProcess != self.thread_count) and count != BATCH_COUNT:
                p = Process(target=self.exportBatch, args=(index,count))
                p.start()
                processes.append(p)
                activeProcess += 1
                count += 1
                index += BATCH_SIZE
            for proc in processes:
                if (not proc.is_alive()):
                    processes.remove(proc)
                    activeProcess -= 1
                        
    def exportBatch(self, index, name):
        try:
            (movements, rates) = testSim.processTimePeriod(self.period_size, index, self.batch_size)
            np.save((os.path.join(self.batchPath, "Batch_" + str(name))), movements)
            np.save((os.path.join(self.labelPath, "Label_" + str(name))), rates)
        except:
            raise ValueError(str(index) +' and '+ str(name))

    def prepareData(self, simulator, reset=False):
        if not os.path.exists(self.batchPath):
            os.makedirs(self.batchPath)
        if not os.path.exists(self.labelPath):
            os.makedirs(self.labelPath)
        if reset:
            for folder in (self.batchPath, self.labelPath):
                for the_file in os.listdir(folder):
                    file_path = os.path.join(folder, the_file)
                    try:
                        if os.path.isfile(file_path):
                            os.unlink(file_path)
                    except:
                        pass
        if "Done" in os.listdir(self.batchPath):
            return
        now = time.time()
        fullData = self.importData(simulator)
        later = time.time()
        print('Total time for all data:{} seconds'.format(int(later-now)))
        count = 0
        f = open(os.path.join(self.batchPath, "Done"), 'w')
        f.close()

    def readFiles(self, fileRangeStart, fileRangeEnd):
        listL = []
        for i in range(fileRangeStart, fileRangeEnd):
            listL.append((str(i), np.load(os.path.join(self.batchPath, "Batch_" + str(i) + ".npy")), np.load(os.path.join(self.labelPath, "Label_"+ str(i) + ".npy"))))
        return listL

In [4]:
testSim = Market(['EUR','USD', 'TRY', 'JPY'], os.path.abspath('../Data_Processing/ProcessedData'))

In [5]:
data = DataManager(batch_size = 50, batch_offset=0)
data.prepareData(testSim, reset=True)



958922
979770
965443
19177
Total time for all data:88 seconds


In [6]:
feedBackup = data.readFiles(0, 1000)

In [7]:
print(feedBackup[0][1])


[[[[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.00024845 1.00035887 1.00024845]
   [1.00034967 1.00034967 1.00034967]
   [1.00031286 1.00044169 1.00031286]
   ...
   [1.00007361 1.00007361 0.99996319]
   [1.0000092  1.00004601 0.9999908 ]
   [1.         1.         0.99988958]]

  [[1.00014329 1.00014329 1.00014329]
   [1.00014036 1.00014036 1.00014036]
   [1.00013744 1.00013744 1.00013744]
   ...
   [0.99999415 0.99999415 0.99999415]
   [0.99999708 0.99999708 0.99999708]
   [1.         1.         1.        ]]

  [[1.00014504 1.00014504 1.00014504]
   [1.00014208 1.00014208 1.00014208]
   [1.00013912 1.00013912 1.00013912]
   ...
   [0.99999408 0.99999408 0.99999408]
   [0.99999704 0.99999704 0.99999704]
   [1.         1.         1.        ]]]


 [[[1.         1.         1.        ]
   [1.      

In [16]:
def main():
    feed = feedBackup.copy()
    seeds = [3, 5, 7]
    tf.reset_default_graph()
    
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    tf.set_random_seed(seeds[0])
    test1 = Agent(len(testSim.currencies), sess)
    sess.run(tf.global_variables_initializer())

    testData = feed[5]
    feed.pop(5)
    #print(testData)
    prices = []
    changes = []
    for data in feed:
        prices.append(data[1])
        changes.append(data[2])
        if np.isinf(data[1]).any() == True:
            print("FOUND in price")
        if np.isinf(data[2]).any() == True:
            print('FOUND in change')
    batch_size = 50
    
    priceMatrix = np.concatenate(prices)
    changeMatrix = np.concatenate(changes)
    
    print(priceMatrix.shape)
    print(changeMatrix.shape)

   
    indexSize = priceMatrix.shape[0]/batch_size
    try:
        for episode in range(50):
            print("Episode: {}".format(episode))
            index = list(range(int(indexSize)))
            batches = len(index)
            loss = 0
            count = len(feed)
            while len(index) != 0:
                for i in random.sample(index, 1):
                    with tf.device('/gpu:0'):
                        addToLoss = test1.train_step(priceMatrix[i*batch_size:i*batch_size + batch_size], changeMatrix[i*batch_size:i*batch_size + batch_size])
                        loss += addToLoss
                        if addToLoss < 0 or addToLoss == np.nan:
                            print('Loss at index {}: {}'.format(i,addToLoss))
                    index.remove(i)
            print(loss/batches)
    except KeyboardInterrupt:
        print("Ending")
        test1.test_model(testData[1], testData[2])
    finally:
        print("Ending")
        test1.test_model(testData[1], testData[2])
    print(testData[2])
            
if __name__ == "__main__":
    main()

(?, 4, 1)
(?, 4, 1, 1)
(49950, 4, 50, 3)
(49950, 4, 1)
Episode: 0
9999.98997825951
Episode: 1
10000.003381310999
Episode: 2
10000.0048730371
Episode: 3
10000.004887700201
Episode: 4
10000.00490138576
Episode: 5
10000.004874992179
Episode: 6
10000.0048828125
Episode: 7
10000.004897475601
Episode: 8
10000.0048925879
Episode: 9
10000.00488965528
Episode: 10
10000.00489552052
Episode: 11
10000.004893565441
Episode: 12
10000.00489552052
Episode: 13
10000.00490138576
Episode: 14
10000.00490431838
Episode: 15
10000.00489552052
Episode: 16
10000.00489454298
Episode: 17
10000.004893565441
Episode: 18
10000.004799721597
Episode: 19
10000.004897475601
Episode: 20
10000.00489845314
Episode: 21
10000.00489552052
Episode: 22
10000.004893565441
Episode: 23
10000.00489454298
Episode: 24
10000.00489552052
Episode: 25
10000.00489649806
Episode: 26
10000.004897475601
Episode: 27
10000.004897475601
Episode: 28
10000.004897475601
Episode: 29
10000.00489649806
Episode: 30
10000.00489649806
Episode: 31
10000